In [8]:
%run 'setup_role_clients.ipynb'
##from setup_role_clients import logstart
##from setup_role_clients import logend

## define logstart and logend
##def logstart(eventname):
##    print(eventname + " started " + str(datetime.datetime.now()))
##def logend(eventname):
##    print(eventname + " ended " + str(datetime.datetime.now()))

## create redshift cluster
logstart("create redshift cluster")
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

## check if cluster is available and keep on checking
logstart("checking cluster availability")
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

while True:
     myclusterprops = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
     if myclusterprops['ClusterStatus'] == 'available':
        break
     else:
        time.sleep(4)

logend("checking cluster availability")

logend("create redshift cluster")

### take note of the cluster endpoint and role arn
prettyRedshiftProps(myclusterprops)

logstart("take note of cluster endpoint and role arn")
DWH_ENDPOINT = myclusterprops['Endpoint']['Address']
DWH_ROLE_ARN = myclusterprops['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)
logend("take note of cluster endpoint and role arn")

## open incoming tcp port to access the cluster endpoint
logstart("open incoming tcp port to access cluster endpoint")
try:
    vpc = ec2.Vpc(id=myclusterprops['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)
logend("open incoming tcp port to access cluster endpoint")

## connect to the cluster
logstart("load ext sql")
%load_ext sql
logend("load ext sql")

logstart("connect to cluster connect")
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string
logend("connect to cluster connect")


create clients for s3 iam redshift started 2023-10-05 04:25:42.408497
create clients for s3 iam redshift ended 2023-10-05 04:25:42.488824
check out sample project data sources on s3 started 2023-10-05 04:25:42.490490
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', ke